# 第2回講義　宿題

## 課題：
* データセット（`text`）で単語の分散表現を学習して、与えられた単語ペア（`word_pairs`）の類似度スコアをcos類似度で計算してください。
* 計算した類似度スコアをsubmission.csvに出力して、submission.csvを提出してください。
* submission.csvのフォーマットはsample_submission.csvを参考にしてください。
* 学習用データセットはtext8というもので、事前に前処理済みの英語のデータセットです。ピリオドもなく文同士がすべて連結した単語の羅列になっているので注意してください。
* text8の全てを学習に使うと時間がかかりすぎるので、今回はその一部(100万単語)を訓練用データセットに使います。
* Word Similarity Taskで評価します。人手で評価した類似度スコアと予測した類似度スコアの相関をPearsonの相関係数によって計算して評価します。
* CBOWやSkipgramなど好きなアルゴリズムを使って構いません。意欲のある方はSubsamplingやHierarchical Softmax([元論文](https://arxiv.org/abs/1310.4546)参照)を実装してみたりsubwordを利用するなどその他の論文を参考にしてみてください。

## 注意：
* 辞書に含める単語の最低出現頻度は**3**としてください。

In [ ]:
# 必要であれば変更してください
import os
os.chdir('/root/userspace/chap2/')

In [ ]:
# データセット概要
with open("./data/text8", "r") as f:
    text = f.readline()
text = text.strip().split()
print("総単語数:", len(text))
print("総単語語彙数:", len(set(text)))
print("データ:", " ".join(text[:100]))
del text

In [ ]:
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PAD = 0 # <PAD>のID
UNK = 1 # <UNK>のID
PAD_TOKEN = '<PAD>' # paddingに使います
UNK_TOKEN = '<UNK>' # 辞書にない単語

word2id = {
    PAD_TOKEN: PAD,
    UNK_TOKEN: UNK,
}

class Vocab(object):
    def __init__(self, word2id={}):
        """
        word2id: 単語(str)をインデックス(int)に変換する辞書
        id2word: インデックス(int)を単語(str)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    

    def build_vocab(self, sentences, min_count=3):
        # 各単語の出現回数の辞書を作成する
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加える
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word

        # 語彙に含まれる単語の出現回数を保持する
        self.raw_vocab = {w: word_counter[w] for w in self.word2id.keys() if w in word_counter}

def load_data():
    with open("./data/text8") as f:
        line = f.readline()
        line = line.strip().split()
    return line

text = load_data()
text = text[:1000000]

vocab = Vocab(word2id=word2id)
vocab.build_vocab([text], min_count=3)

In [ ]:
word_pairs = []
with open("./data/sample_submission.csv", "r") as fin:
    for line in fin:
        line = line.strip().split(",")
        word1 = line[0]
        word2 = line[1]
        word_pairs.append([word1, word2])

In [ ]:
######################## 解答例 ##############################

def sentence_to_ids(vocab, sen):
    """
    単語のリストをIDのリストに変換する関数
    :param vocab: Vocab
    :param sen: list of str, 文を分かち書きして得られた単語のリスト
    :return out: list of int, 単語IDのリスト
    """
    out = [vocab.word2id.get(word, UNK) for word in sen] # 辞書にない単語にはUNKのIDを割り振ります
    return out

id_text = [sentence_to_ids(vocab, sen) for sen in text]

def pad_seq(seq, max_length):
    """
    :param seq: list of int, 単語のインデックスのリスト
    :param max_length: int, バッチ内の系列の最大長
    :return seq: list of int, 単語のインデックスのリスト
    """
    seq += [PAD for i in range(max_length - len(seq))]
    return seq

# Hyper Parameters
batch_size = 64 # ミニバッチのサイズ
max_batch = 500 # 今回学習するミニバッチの数
num_embeddings = len(vocab.word2id) # 語彙の総数
embedding_dim = 300 # 各単語に割り当てるembeddingの次元数

class DataLoader_CBOW(object):
    def __init__(self, text, batch_size, window=3):
        """
        :param text: list of list of int, 単語をIDに変換したデータセット
        :param batch_size: int, ミニバッチのサイズ
        :param window: int, 周辺単語とターゲットの単語の最大距離
        """
        self.text = text
        self.batch_size = batch_size
        self.window = window
        self.s_pointer = 0 # データセット上を走査する文単位のポインター
        self.w_pointer = 0 # データセット上を走査する単語単位のポインター
        self.max_s_pointer = len(text) # データセットに含まれる文の総数

    def __iter__(self):
        return self

    def __next__(self):
        batch_x = []
        batch_y = []
        while len(batch_x) < self.batch_size:
            # 走査する対象の文
            sen = self.text[self.s_pointer]

            # 入力となる単語群を取得
            start = max(0, self.w_pointer - self.window)
            word_x = sen[start:self.w_pointer] + \
                sen[self.w_pointer + 1:self.w_pointer + self.window + 1]
            word_x = pad_seq(word_x, self.window * 2)

            # 予測すべき単語
            word_y = sen[self.w_pointer]

            batch_x.append(word_x)
            batch_y.append(word_y)
            self.w_pointer += 1

            if self.w_pointer >= len(sen):
                # 文を走査し終わったら次の文の先頭にポインターを移行する
                self.w_pointer = 0
                self.s_pointer += 1
                if self.s_pointer >= self.max_s_pointer:
                    # 全ての文を走査し終わったら終了する
                    self.s_pointer = 0
                    raise StopIteration

        # データはtorch.Tensorにする必要があります。dtype, deviceも指定します。
        batch_x = torch.tensor(batch_x, dtype=torch.long, device=device)
        batch_y = torch.tensor(batch_y, dtype=torch.long, device=device)

        return batch_x, batch_y

class CBOW(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(CBOW, self).__init__()
        """
        :param num_embeddings: int, 語彙の総数
        :param embedding_dim: int, 単語埋め込みベクトルの次元
        """
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim

        # 埋め込み層
        self.embedding = nn.Embedding(self.num_embeddings, self.embedding_dim)
        # 全結合層(バイアスなし)
        self.linear = nn.Linear(self.embedding_dim, self.num_embeddings, bias=False)

        # xavier初期化でパラメータを初期化
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, word_x, word_y):
        """
        :param word_x: torch.Tensor(dtype=torch.long), (batch_size, window*2)
        :param word_y: torch.Tensor(dtype=torch.long), (batch_size,)
        """
        emb_x = self.embedding(word_x) # (batch_size, window*2, embedding_dim)
        # paddingした部分を無視するためにマスクをかけます
        emb_x = emb_x * (word_x != PAD).float().unsqueeze(-1) # (batch_size, window*2, embedding_dim)
        sum_x = torch.sum(emb_x, dim=1) # (batch_size, embedding_dim)
        lin_x = self.linear(sum_x) # (batch_size, num_embeddings)
        log_prob_x = F.log_softmax(lin_x, dim=-1) # (batch_size, num_embeddings)
        loss = F.nll_loss(log_prob_x, word_y)
        return loss

def compute_loss(model, inputs, optimizer=None, is_train=True):
    # is_train=Trueならモデルをtrainモードに、
    # is_train=Falseならモデルをevaluationモードに設定します
    model.train(is_train)

    # lossを計算します。
    loss = model(*inputs)

    if is_train:
        # .backward()を実行する前にmodelのparameterのgradientを全て0にセットします
        optimizer.zero_grad()
        # parameterのgradientを計算します。
        loss.backward()
        # parameterのgradientを用いてparameterを更新します。
        optimizer.step()

    return loss.item()

# モデル
cbow = CBOW(num_embeddings, embedding_dim).to(device)
# optimizer
optimizer_cbow = Adam(cbow.parameters())
# データローダー
dataloader_cbow = DataLoader_CBOW(id_text, batch_size)

for epoch in range(1, 2):
    start_at = time.time()
    for batch, (x, y) in enumerate(dataloader_cbow):
        loss = compute_loss(cbow, (x, y), optimizer=optimizer_cbow, is_train=True)
        if batch % 1000 == 0:
            print("Batch:{}, Loss:{}".format(batch, loss))
    end_at = time.time()
    elapsed_time = end_at - start_at
    m = int(elapsed_time // 60)
    s = elapsed_time % 60

    print("Epoch: {}, Loss: {:.4f}, Elapsed time: {}m {:.2f}s".format(epoch, loss, m, s))

embedding_matrix = cbow.embedding.weight.data.cpu().numpy()

pred_scores = []
for pair in word_pairs:
    w1 = embedding_matrix[vocab.word2id[pair[0]]]
    w2 = embedding_matrix[vocab.word2id[pair[1]]]
    score = np.dot(w1, w2)/np.linalg.norm(w1, ord=2)/np.linalg.norm(w2, ord=2)
    pred_scores.append(score)

with open("./data/submission.csv", "w") as fout:
    for pair, score in zip(word_pairs, pred_scores):
        fout.write(pair[0] + "," + pair[1] + "," + str(score) + "\n")